##Workforce Optimization: From Demand to Supply

The Problem

You are managing a customer support operation. Every 30 minutes, you receive forecasts.
- Demand: Expected volume of customer contacts (e.g., 500 calls)
- AHT: Average handle time per contact (e.g., 180 seconds)
- Constraints: Service level agreements (e.g., 80% of calls answered within 20 seconds)

Your Task: Determine the minimum number of agents (supply) needed to meet service levels while minimizing cost.


Why This Is Hard

This is NOT a simple Erlang-C lookup problem. Real-world complications include:
Multiple skill groups with different handle times
- Agent shrinkage (breaks, meetings, training) reducing effective capacity
- Occupancy limits (agents cannot work at 100% utilization - they burn out)
- Routing rules (which queues can be handled by which agent groups)
- Time-varying demand across 48 intervals per day
______________________________________________________________________________
The Mathematics: Two Complementary Approaches
1. Erlang-C: The Performance Model
Erlang-C answers: "Given N agents, volume V, and handle time H, what is the wait time and occupancy?"
Key Insight: Erlang-C is a constraint checker, not an optimizer. It tells you if a solution is feasible, but not what the optimal solution is.
2. Linear/Nonlinear Programming: The Optimization Framework
This is where we find the minimum cost solution subject to constraints:
minimize:    Sum of headcount_i              (minimize total workforce)

subject to:  ASA(headcount) <= patience      (service level constraint)
             occupancy(headcount) <= max_occ (utilization constraint)
             headcount_min <= headcount <= max (capacity bounds)
Key Insight: The objective function is simple (minimize headcount), but the constraints are nonlinear because ASA and occupancy are computed via Erlang-C formulas.
______________________________________________________________________________
Your Learning Path
Phase 1: Understand the Individual Components

1. Review Erlang-C deeply
Implement Erlang-C from scratch in Python
Understand the Pw (probability of wait) formula and why it is nonlinear

2. Learn Constrained Optimization basics
Start with linear programming (LP) using scipy.optimize.linprog
Solve simple problems: resource allocation, diet problem, production planning

Phase 2: Bridge the Gap

3. Understand Nonlinear Optimization
Graduate from LP to constrained nonlinear optimization
Learn about feasible regions, active constraints, and local vs. global optima
Tools: scipy.optimize.minimize with constraints
Try: Minimize headcount with a simplified Erlang-C constraint
4. Study Real Solvers
Look at golden section search for single-variable problems
Understand why gradient-free methods are used (Erlang-C has no closed-form gradient)

Phase 3:

Find: Minimum headcount of agents needed given the demand data

Requirements:
Implement Erlang-C calculation
Use scipy.optimize.minimize or write a simple search algorithm
Print: optimal headcount, resulting ASA, resulting occupancy
Validate: manually verify your Erlang-C outputs against online calculators

______________________________________________________________________________
The Big Picture
This problem combines:
Queueing Theory (Erlang-C) - to model system behavior
Optimization Theory (LP/NLP) - to find the best solution
Software Engineering - to make it fast and scalable

You are not just learning math; you are learning how to engineer solutions to real-world operations problems that companies like Amazon, Uber, and call centers use every day.
______________________________________________________________________________
Resources
Erlang-C Calculator (to verify): https://www.erlang.com/calculator/

SciPy Optimization Docs: https://docs.scipy.org/doc/scipy/reference/optimize.html

https://www.gurobi.com/case_studies/air-france-tail-assignment-optimization/

https://www.youtube.com/watch?v=E72DWgKP_1Y

The Art of Linear Programming: https://www.youtube.com/watch?v=E72DWgKP_1Y



In [ ]:
import math

import numpy as np
import pandas as pd


def erlang_c_P_wait(offered_load, agents):
    a = offered_load
    N = agents
    if N <= 0:
        return 1.0
    if a >= N:
        return 1.0
    sum_terms = sum(a**k / math.factorial(k) for k in range(0, N))
    last = a**N / math.factorial(N)
    Pw = (last * N / (N - a)) / (sum_terms + last * N / (N - a))
    return Pw


def erlang_c_Wq(offered_load, agents, service_rate):
    a = offered_load
    N = agents
    lambda_rate = a * service_rate
    if N * service_rate - lambda_rate <= 0:
        return float("inf")
    Pw = erlang_c_P_wait(a, N)
    Wq = Pw / (N * service_rate - lambda_rate)
    return Wq


def service_level_within_threshold(
    offered_load, agents, service_rate, threshold_seconds
):
    a = offered_load
    N = agents
    lambda_rate = a * service_rate
    if a >= N:
        return 0.0
    Pw = erlang_c_P_wait(a, N)
    decay = math.exp(-(N * service_rate - lambda_rate) * threshold_seconds)
    return 1.0 - Pw * decay


def occupancy(offered_load, agents):
    if agents == 0:
        return 1.0
    return offered_load / agents


def required_scheduled_agents_for_interval(
    demand_contacts,
    aht_seconds,
    interval_seconds=1800,
    shrinkage=0.35,
    sla_target=0.80,
    sla_threshold_seconds=20.0,
    max_occupancy=0.85,
    search_max=500,
):
    lambda_per_sec = demand_contacts / interval_seconds
    mu = 1.0 / aht_seconds
    a = lambda_per_sec / mu
    for scheduled in range(0, search_max + 1):
        available = scheduled * (1.0 - shrinkage)
        available_int = max(0, int(math.floor(available + 1e-9)))
        if available_int == 0:
            if demand_contacts == 0:
                return {
                    "scheduled": scheduled,
                    "available": available_int,
                    "Pw": 0.0,
                    "Wq": 0.0,
                    "service_level": 1.0,
                    "occupancy": 0.0,
                    "note": None,
                }
            else:
                continue
        if a >= available_int:
            continue
        sl = service_level_within_threshold(
            a, available_int, mu, sla_threshold_seconds
        )
        occ = occupancy(a, available_int)
        if sl >= sla_target and occ <= max_occupancy:
            Pw = erlang_c_P_wait(a, available_int)
            Wq = erlang_c_Wq(a, available_int, mu)
            return {
                "scheduled": scheduled,
                "available": available_int,
                "Pw": Pw,
                "Wq": Wq,
                "service_level": sl,
                "occupancy": occ,
                "note": None,
            }
    return None


# synthetic demand
np.random.seed(0)
intervals = 48
base = 50
t = np.arange(intervals)
demand_pattern = (
    base
    + 80 * np.exp(-((t - 12) ** 2) / (2 * 5**2))
    + 60 * np.exp(-((t - 36) ** 2) / (2 * 6**2))
)
demand_pattern = np.round(
    demand_pattern + np.random.normal(scale=5, size=intervals)
).astype(int)

aht_seconds = 180.0
shrinkage = 0.30
sla_target = 0.80
sla_threshold_seconds = 20.0
max_occupancy = 0.85

results = []
total_scheduled = 0
for i, demand in enumerate(demand_pattern):
    res = required_scheduled_agents_for_interval(
        demand,
        aht_seconds,
        interval_seconds=1800,
        shrinkage=shrinkage,
        sla_target=sla_target,
        sla_threshold_seconds=sla_threshold_seconds,
        max_occupancy=max_occupancy,
        search_max=800,
    )
    if res is None:
        lambda_per_sec = demand / 1800.0
        mu = 1.0 / aht_seconds
        a = lambda_per_sec / mu
        avail_needed = int(math.ceil(a / max_occupancy))
        scheduled_needed = int(math.ceil(avail_needed / (1.0 - shrinkage)))
        res = {
            "scheduled": scheduled_needed,
            "available": max(
                0, int(math.floor(scheduled_needed * (1.0 - shrinkage)))
            ),
            "Pw": None,
            "Wq": None,
            "service_level": None,
            "occupancy": None,
            "note": "no feasible solution within search_max",
        }
    results.append({"interval": i, "demand": demand, **res})
    total_scheduled += res["scheduled"]

df = pd.DataFrame(results)
df["service_level"] = df["service_level"].round(4)
df["occupancy"] = df["occupancy"].round(4)
df["Wq_seconds"] = df["Wq"].apply(
    lambda x: round(x, 3) if x is not None else None
)
df = df[
    [
        "interval",
        "demand",
        "scheduled",
        "available",
        "service_level",
        "Wq_seconds",
        "occupancy",
        "Pw",
        "note",
    ]
]

Total scheduled agents across 48 intervals (sum of scheduled headcount): 873

Top 5 intervals by demand and scheduled agents:
 interval  demand  scheduled  available  service_level  Wq_seconds  occupancy       Pw note
       11     136         25         17         0.8002      15.433     0.8000 0.291504 None
       12     134         25         17         0.8220      13.280     0.7882 0.265592 None
       13     129         25         17         0.8683       9.119     0.7588 0.207721 None
       14     126         23         16         0.8100      14.679     0.7875 0.277266 None
       10     125         23         16         0.8210      13.583     0.7812 0.264105 None
